In [1]:
import sys
import os

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import time

Using TensorFlow backend.


In [ ]:
start = time.time()

In [2]:
DEV = False
argvs = sys.argv
argc = len(argvs)

In [3]:
if argc > 1 and(argvs[1] == "--development" or argvs[1] == "-d"):
    DEV = True
    
if DEV:
    epochs = 2
else:
    epochs = 30
    
train_data_path = 'data/train'
validation_data_path = 'data/test'

### Parameters

In [4]:
img_width, img_height = 150, 150
batch_size = 32
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 2
lr = 0.0004

In [5]:
model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

W0301 19:50:41.607834 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,..., padding="same")`
  
W0301 19:50:41.628747 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 19:50:41.632766 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 19:50:41.651720 19680 deprecation_wrapper.py:119] 

In [6]:
model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=lr),
             metrics=['accuracy'])

W0301 19:50:42.223713 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        8256      
_________________________________________________________________
activation_2 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 88800)             0         
__________

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1. /255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [9]:
test_datagen = ImageDataGenerator(rescale=1. /255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 470 images belonging to 2 classes.


In [11]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 30 images belonging to 2 classes.


### Tensorboard log

In [12]:
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]

In [13]:
model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)

C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=30, validation_data=<keras_pre..., callbacks=[<keras.ca..., validation_steps=300, steps_per_epoch=31)`
  import sys
W0301 19:50:50.433515 19680 deprecation.py:323] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0301 19:50:50.942562 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3\envs\fhadli\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0301 19:50:50.943596 19680 deprecation_wrapper.py:119] From C:\Users\LENOVO\Anaconda3

Epoch 1/30
30/31 [============================>.] - ETA: 0s - loss: 7.7913 - acc: 0.5005

KeyboardInterrupt: 

In [14]:
target_dir = './models/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

In [16]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir logs/fit

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 18108.